In [ ]:
import sqlite3
from sqlite3 import Error

# Creates database connection to an SQLite database.
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

# Creates Students, Enrolled, and Courses tables at the specified database connection, if they don't exist already.
def create_tables(conn):
    if conn is not None:
        sql_create_students_table = """ CREATE TABLE IF NOT EXISTS Students (
                                            sid integer PRIMARY KEY,
                                            sname text NOT Null,
                                            age integer);"""
        sql_create_enrolled_table = """CREATE TABLE IF NOT EXISTS Enrolled (
                                            sid integer,
                                            cid integer,
                                            grade text,
                                            FOREIGN KEY (sid) REFERENCES Students (sid)
                                            FOREIGN KEY (cid) REFERENCES Courses (cid)
                                            );"""
        sql_create_courses_table = """ CREATE TABLE IF NOT EXISTS Courses (
                                            cid integer PRIMARY KEY,
                                            cname text,
                                            credits integer);"""
        try:
            c=conn.cursor()
            c.execute(sql_create_students_table)
            c.execute(sql_create_enrolled_table)
            c.execute(sql_create_courses_table)
        except Error as e:
            print(e)
    else:
        print("Error! Can not create the database connection.")

# Creates a new Student record and adds it to the Student table.
def create_student(conn, values):
    sql = ''' INSERT INTO Students(sid, sname, age) VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

# Creates a new Enrollment record and adds it to the Enrolled table.
def create_enrollment(conn, values):
    sql = ''' INSERT INTO Enrolled(sid, cid, grade) VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

# Creates a new Course record and adds it to the Course table.
def create_course(conn, values):
    sql = ''' INSERT INTO Courses(cid, cname, credits) VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

# Populates all tables with sample data.
def populate_tables_with_sample_data(conn):
    # Add sample students.
    student_1 = ('1', 'Hugh G. Rection', '15')        
    create_student(conn, student_1)
    
    # Add sample courses.
    course_1 = ('313', 'Fundamentals of Programming Languages', '3')
    course_2 = ('315', 'Software Engineering I', '3')
    course_3 = ('355', 'Database Systems', '3')
    create_course(conn, course_1)
    create_course(conn, course_2)
    create_course(conn, course_3)
    
    # Add sample enrollments.
    enrollment_1 = ('1', '355', 'A')
    create_enrollment(conn, enrollment_1)

# Begins the command line interface.
def start_cmd_interface(conn):
    # Prompt user to sign in by student id or create new student record by typing -1.
    sid = input("Enter your student id: ")
    
    if(sid == "-1"):
        create_new_student(conn)
        start_cmd_interface(conn)
    else:
        # Prompt user to select a function and execute the appropriate function.
        userInput = ''
        while(userInput != 'X'):
            userInput = input("Enter a command: ")
            if(userInput == 'L'):
                select_all_courses(conn)
            elif(userInput == 'E'):
                enroll_in_course(conn, sid)
            elif(userInput == 'W'):
                withdraw_from_course(conn, sid)
            elif(userInput == 'S'):
                search_for_course(conn)
            elif(userInput == 'M'):
                view_my_classes(conn, sid)
            elif(userInput == 'H'):
                show_help_prompt()
            elif(userInput == 'X'):
                print("Goodbye!")
            else:
                print("Not a valid command.")

# Prompts the user to enter their personal information to create a new student record in the Students table.
def create_new_student(conn):

    sid = input("Enter an sid\(DEBUG\):")
    #Constraint Block
    cur = conn.cursor()
    query = "SELECT count(*) FROM Students WHERE sid = ?"
    while (True):
        if(cur.execute(query, (sid,)).fetchall()[0] < (1,)):
            print("Registering New Student")
            break
        else:
            print("StudentID is already taken.")
            sid = input("Please enter a new sid\(DEBUG\):")
    #Constraint Block Ends
    sname = input("Enter your name: ")
    age = input("Enter your age: ")
    values = (sid, sname, age)
    create_student(conn, values)
    conn.commit()

# Prints all records in the Courses table.
def select_all_courses(conn):
    cur = conn.cursor()
    cur.execute("SELECT * FROM Courses")
    rows = cur.fetchall()
    for row in rows:
        print(row)

# Enrolls a student in a course.
def enroll_in_course(conn, sid):
    cid = input("Enter a valid course id: ")
    cur = conn.cursor()
    #Constraint Block
    query = "SELECT count(*) FROM Courses WHERE cid = ?"
    sec_query = "SELECT count(*) FROM Enrolled WHERE cid = ?"
    while (True):
        if(cur.execute(query, (cid,)).fetchall()[0] > (0,) and cur.execute(sec_query,(cid,)).fetchall()[0] <(1,)):
            print("Registering for Course")
            break
        else:
            print("Entered Course ID was Invalid")
            cid = input("Please enter a valid course id: ")
        
    #Constraint Block End
    values = (sid, cid, '')
    create_enrollment(conn, values)
    conn.commit()

# Withdraws a student from a course.
def withdraw_from_course(conn, sid):
    course_id = input("Enter a valid course id: ")
    cur = conn.cursor()
    #Constraint Block
    
    query = "SELECT count(*) FROM Courses WHERE cid = ?"
    sec_query = "SELECT count(*) FROM Enrolled WHERE cid = ?"
    while (True):
        if (cur.execute(query, (course_id,)).fetchall()[0]>(0,)  and cur.execute(sec_query, (course_id,)).fetchall()[0]>(0,)):
            print("Withdrawing from Course")
            break
        else:
            print("Entered Course ID is invalid.")
            course_id = input("Enter a valid course id: ")
    
    #Constraint Block End
                
    values = (sid, course_id)
    sql = 'DELETE FROM Enrolled WHERE sid = ? AND cid = ?'
    #cur = conn.cursor()
    cur.execute(sql, values)
    conn.commit()

# Prompts the user to enter a substring to return courses with names similar to the substring.
def search_for_course(conn):
    substring = input("Enter a substring: ")
    sql = 'SELECT * FROM Courses WHERE cname LIKE \"%?%\"'
    cur = conn.cursor()
    cur.execute(sql.replace("?", substring))
    rows = cur.fetchall()
    for row in rows:
        print(row)

# Prints all courses a student is enrolled in.
def view_my_classes(conn, sid):
    sql = 'SELECT c.cid, c.cname, c.credits FROM Enrolled e, Courses c WHERE e.cid = c.cid AND e.sid = ?'
    cur = conn.cursor()
    cur.execute(sql.replace("?", sid))
    rows = cur.fetchall()
    for row in rows:
        print(row)

# Shows a list of functions the user can execute in the command line interface.
def show_help_prompt():
    print("L: List all available courses",
          "\nE: Enroll yourself in a course",
          "\nW: Withdraw yourself from a course",
          "\nS: Search for a course by name",
          "\nM: List your current enrollments",
          "\nH: List executable functions",
          "\nX: Exit application")

# Main method.
def main():
    
    # Create connection to database.
    database = r"DatabaseProject.db"
    conn = create_connection(database)
    
    # Create Students, Enrolled, and Courses tables, if they don't already exist.
    create_tables(conn)
    
    # Populate the tables with sample data...
    # populate_tables_with_sample_data(conn)
    
    # Begin command line interface.
    start_cmd_interface(conn)
    
    # Close connection to database.
    conn.close()

if __name__ == '__main__':
     main()

Enter your student id: -1
Enter an sid\(DEBUG\):1
StudentID is already taken.
Please enter a new sid\(DEBUG\):1
StudentID is already taken.
Please enter a new sid\(DEBUG\):1
StudentID is already taken.
Please enter a new sid\(DEBUG\):2
StudentID is already taken.
Please enter a new sid\(DEBUG\):3
StudentID is already taken.
Please enter a new sid\(DEBUG\):4
Registering New Student
